In [2]:
from importlib import reload
import matplotlib.pyplot as plt
import dnd

from dnd import Dice
from creatures import get_creatures_df, creature_damage_save, creature_condition_save, get_resistance_stats, creature_damage_roll

import pandas as pd
import numpy as np

creatures = get_creatures_df()
sources = ['MM', 'ToB1-2023', 'MPMM', 'MTF']
creatures = creatures[creatures['source'].isin(sources)].reset_index(drop=True)

In [3]:
dnd.fundamental_math()

,Level_CR,Abl,Prof,Total,AC,Hit_Pcnt
0,1,3,2,5,13,65
1,2,3,2,5,13,65
2,3,3,2,5,13,65
3,4,4,2,6,14,65
4,5,4,3,7,15,65
5,6,4,3,7,15,65
6,7,4,3,7,15,65
7,8,5,3,8,16,65
8,9,5,4,9,16,70
9,10,5,4,9,17,65


In [4]:
# simulate damage per round with a paladin level 10 character with a +3 weapon and a 2 level dip to warlock
level = 10
charisma = 4
agonizing_blast = charisma
proficiency_bonus = dnd.proficency_bonus(level)
hex = Dice(1, 6)
hex_warrior = proficiency_bonus
weapon_damage = Dice(1, 10)
weapon_bonus_damage = 3 + charisma + hex_warrior
to_hit = 10

ac = dnd.get_AC(level)

smite = Dice(2, 8)


num_attacks = 3

adv = {'advantage':True, 'crit_on':20}

print('ts smite hw\t\t',
    dnd.roll_damage(ac, to_hit, weapon_damage, 
    bonus_damage=weapon_bonus_damage , **adv
    , crit_added_damage_dice=smite
).mean()*num_attacks)



ts smite hw		 49.427099999999996


In [5]:
level = 10
charisma = 3
agonizing_blast = charisma
proficiency_bonus = dnd.proficency_bonus(level)
hex = Dice(1, 6)
hex_warrior = proficiency_bonus
weapon_damage = Dice(1, 10)
weapon_bonus_damage = 3 + charisma + charisma + hex_warrior
to_hit = 10
# get dc
ac = dnd.get_AC(level)

smite = Dice(2, 8)


num_attacks = 3

adv = {'advantage':True, 'crit_on':20}


print('ts smite hw\t\t',
    dnd.roll_damage(ac, to_hit, weapon_damage, 
    bonus_damage=weapon_bonus_damage , **adv
    , crit_added_damage_dice=smite
).mean()*num_attacks)


ts smite hw		 54.9564


In [37]:
level = 20
charisma = 5
agonizing_blast = charisma
proficiency_bonus = dnd.proficency_bonus(level)
hex = Dice(1, 6)
hex_warrior = proficiency_bonus
weapon_damage = Dice(1, 10)
weapon_bonus_damage = 3 + charisma + charisma

smite = Dice(5, 8)
eldritch_smite = Dice(5, 8)


eldritch_blast = Dice(1, 10)

num_attacks = 3
num_eb = 4

adv = {'advantage':False, 'crit_on':20}

print('Eldritch blast \t\t\t\t', dnd.roll_damage(20, 13, eldritch_blast+hex, bonus_damage=agonizing_blast).mean()*num_eb)

print('Eldritch blast + hex warrior\t\t', dnd.roll_damage(20, 13, eldritch_blast, bonus_damage=agonizing_blast+hex_warrior, **adv).mean()*num_eb)

smite_on_hit_dpr = dnd.roll_damage(20, 14, Dice(1, 10), 
    bonus_damage=weapon_bonus_damage+hex_warrior, crit_added_damage_dice=smite, **adv
).mean()*num_attacks

print('+3 weapon + hex warrior + PAM (smite on hite)\t', smite_on_hit_dpr)

print('+3 weapon + hex warrior + PAM (smite on crit)\t',
    dnd.roll_damage(20, 14, Dice(1, 10)+smite, 
    bonus_damage=weapon_bonus_damage+hex_warrior, **adv
).mean()*num_attacks)


Eldritch blast 				 40.9128
Eldritch blast + hex warrior		 47.9192
+3 weapon + hex warrior + PAM (smite on hite)	 59.8335
+3 weapon + hex warrior + PAM (smite on crit)	 110.01929999999999


In [46]:
# oathbreaker paladins can summon skeletons
# dante the oathbreaker is going to get to create a lot of summons by using warlock spell slots

num_slots = 2
number_of_hours_sleeping = 8
anmiate_dead_skeletons_at_5th_level = 5

#no sleep till army of the dead
number_of_skeletons =  num_slots  * number_of_hours_sleeping * anmiate_dead_skeletons_at_5th_level


num_melee_skeletons = 4
num_ranged_skeletons = number_of_skeletons - num_melee_skeletons

melee_skeleton_damage = (dnd.roll_damage(20, 4+3, Dice(1, 6), 
    bonus_damage=3+5 
).mean() + dnd.roll_damage(20, 0+3, Dice(1, 6), 
    bonus_damage=3+5 
).mean())*num_melee_skeletons

# dual wielding skeletons with +3 weapons (8 +3 weapons needed)
print(f'{num_melee_skeletons} melee sele attack \t', melee_skeleton_damage)

ranged_skeleton_damage = dnd.roll_damage(20, 4, Dice(1, 6), 
    bonus_damage=0, crit_on=20
).mean()*num_ranged_skeletons

# bow an arrow skeletons with normal weapons
print(f'{num_ranged_skeletons} ranged skeleton attacks \t', ranged_skeleton_damage)

4 melee sele attack 	 29.0816
76 ranged skeleton attacks 	 78.394


In [45]:
print('potential dpr of dante + melee skeletons\t', smite_on_hit_dpr + melee_skeleton_damage )

print('potential dpr of dante + all spoky bois\t\t', smite_on_hit_dpr + melee_skeleton_damage + ranged_skeleton_damage)

print()
print('Reasonable damage for level 20')
dpr = dnd.damage_targets().query("Level_CR==20")[['High_DPR', 'Target_DPR']].mean()
print(dpr)



potential dpr of dante + melee skeletons	 88.2375
potential dpr of dante + all spoky bois		 165.96269999999998

Reasonable damage for level 20
High_DPR      66.7
Target_DPR    33.3
dtype: float64
